<h1>Win Predictor Data Analysis</h1>
In this notebook, we will try to understand the data that we will be using for the cricket win predictor and also analyze it for predictive features.

In [304]:
#Load the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

<h2>Data Cleaning</h2>
First we will do some data cleaning and extract the features that we want.

In [378]:
# load the data and inspect it
win_data = pd.read_pickle(r"../Resources/ground_player_lineup.pkl")
win_data.rename(columns={'Winner Name':'winner name'}, inplace = True)
win_data.columns = win_data.columns.droplevel(0)
win_data.reset_index(inplace = True)
win_data.rename(columns={'Match Code':'match code'}, inplace = True)
win_data.head()

,match code,Ground Name,winner name,Batting Team,Bowling Team,P1,P2,P3,P4,P5,...,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11
0,1381452,"Sportpark Het Schootsveld, Deventer",Germany,Austria,Germany,M Simpson-Parker,Iqbal Hossain,Razmal Shigiwal,A Randhawa,Shahil Momin,...,J van Heerden,MJ Richardson,Faisal Mubashir,Ghulam Ahmadi,Muslim Yar,SVE Bharathi,Abdul Bashir,Sachin Mandy,Sahir Naqash,H Singh
1,1381451,"Sportpark Het Schootsveld, Deventer",Germany,Austria,Germany,M Simpson-Parker,Iqbal Hossain,Razmal Shigiwal,A Randhawa,Mirza Ahsan,...,J van Heerden,VB Chikkannaiah,Faisal Mubashir,Sahir Naqash,DA Blignaut,Ghulam Ahmadi,Sachin Mandy,SVE Bharathi,Muslim Yar,D Klein
2,1381456,"National Sports Academy, Sofia",Turkey,Croatia,Turkey,WK Bitis,Naseem Khan,P Bosnjak,Sohail Ahmad,A Daxini,...,Ilyas Ataullah,A Turkmen,R Nath,Muhammed Turkmen,M Ozturk,Cihan Altun,Muhammet Bicer,Muhammet Kursat,Murat Ipek,Murat Yilmaz
3,1381457,"National Sports Academy, Sofia",Serbia,Croatia,Serbia,A Daxini,Naseem Khan,P Bosnjak,V Zanko,A Magdalenic,...,A Dizija,LA Dunbar,MV Pavlovic,S Tosic,V Zimonjic,N Zimonjic,NR Johns-Wickberg,Rahman Ademi,AA Gajic,M Sarenac
4,1381458,"National Sports Academy, Sofia",Bulgaria,Turkey,Bulgaria,Ilyas Ataullah,I Elec,A Turkmen,R Nath,M Ozturk,...,Zain Asif,Zaid Soulat,P Mishra,Isa Zaroo,D Nikolov,Danyal Raja,A Ahmadhel,Gagandeep Singh,DV Varghese,Huzaif Yousuf


In [379]:
# Load the match data and inspect it
match_data = pd.read_csv(r"../Resources/match_data.csv")
match_data.drop(columns =['batter','bowler','non_striker','batter runs','extra runs','total runs','extras','wickets','out','non_boundary','review','replacements'], inplace = True)
match_data.head()

C:\Users\athar\AppData\Local\Temp\ipykernel_2572\662367770.py:2: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  match_data = pd.read_csv(r"../Resources/match_data.csv")


,match code,country,over,ball,score,total wickets
0,1381452,Austria,0,0,0,0
1,1381452,Austria,0,1,4,0
2,1381452,Austria,0,2,4,0
3,1381452,Austria,0,3,4,0
4,1381452,Austria,0,4,5,0


In [380]:
# Merge the two tables to get the data we need
win_pred_data = match_data.merge(win_data[['match code','winner name']], on =['match code'])
win_pred_data.head()

,match code,country,over,ball,score,total wickets,winner name
0,1381452,Austria,0,0,0,0,Germany
1,1381452,Austria,0,1,4,0,Germany
2,1381452,Austria,0,2,4,0,Germany
3,1381452,Austria,0,3,4,0,Germany
4,1381452,Austria,0,4,5,0,Germany


In [381]:
# Make a column that shows if a country won
win_pred_data['win'] = (win_pred_data['country'] == win_pred_data['winner name']).astype(int)
win_pred_data['final score'] = win_pred_data.groupby(['match code', 'country'])['score'].transform('last')
win_pred_data.head()

,match code,country,over,ball,score,total wickets,winner name,win,final score
0,1381452,Austria,0,0,0,0,Germany,0,121
1,1381452,Austria,0,1,4,0,Germany,0,121
2,1381452,Austria,0,2,4,0,Germany,0,121
3,1381452,Austria,0,3,4,0,Germany,0,121
4,1381452,Austria,0,4,5,0,Germany,0,121


In [382]:
# Add a column that shows the target the second country has to make
win_pred_data['target'] = win_pred_data.groupby('match code')['final score'].transform('first')
win_pred_data['target'] += 1

# Add a column that shows which the first country is
win_pred_data['first country'] = win_pred_data.groupby('match code')['country'].transform('first')
win_pred_data.head()

,match code,country,over,ball,score,total wickets,winner name,win,final score,target,first country
0,1381452,Austria,0,0,0,0,Germany,0,121,122,Austria
1,1381452,Austria,0,1,4,0,Germany,0,121,122,Austria
2,1381452,Austria,0,2,4,0,Germany,0,121,122,Austria
3,1381452,Austria,0,3,4,0,Germany,0,121,122,Austria
4,1381452,Austria,0,4,5,0,Germany,0,121,122,Austria


In [383]:
# Remove all those rows where country == first country
win_pred_data = win_pred_data[(win_pred_data['country'] != win_pred_data['first country'])]
win_pred_data.head()

,match code,country,over,ball,score,total wickets,winner name,win,final score,target,first country
123,1381452,Germany,0,0,2,0,Germany,1,123,122,Austria
124,1381452,Germany,0,1,3,0,Germany,1,123,122,Austria
125,1381452,Germany,0,2,4,0,Germany,1,123,122,Austria
126,1381452,Germany,0,3,4,0,Germany,1,123,122,Austria
127,1381452,Germany,0,4,6,0,Germany,1,123,122,Austria


In [384]:
# We are also going to need the other country's rating so load the data
country_data = pd.read_csv(r"../Resources/country_data.csv")
country_data.head()

,position,country,matches,points,rating
0,1,India,54,14267,264
1,2,England,39,10117,259
2,3,New Zealand,46,11794,256
3,4,Pakistan,50,12719,254
4,5,South Africa,32,8080,253


In [385]:
# Merge the rating with win_pred_data for the first country
win_pred_data = win_pred_data.merge(country_data[['rating', 'country']], left_on='first country', right_on='country', how='inner')

In [386]:
win_pred_data.drop(columns=['winner name', 'final score', 'first country', 'country_y'], inplace = True)
win_pred_data.rename(columns={'rating':'other rating', 'country_x':'country'}, inplace = True)
win_pred_data.head()

,match code,country,over,ball,score,total wickets,win,target,other rating
0,1381452,Germany,0,0,2,0,1,122,58
1,1381452,Germany,0,1,3,0,1,122,58
2,1381452,Germany,0,2,4,0,1,122,58
3,1381452,Germany,0,3,4,0,1,122,58
4,1381452,Germany,0,4,6,0,1,122,58


In [387]:
# Now finally remove the extra balls and we are done and add the playing country's rating
win_pred_data = win_pred_data.merge(country_data[['country','rating']], on = ['country'])
win_pred_data['balls'] = win_pred_data['over'] * 6 + win_pred_data['ball']
win_pred_data.sort_values(by=['match code','balls'])
win_pred_data = win_pred_data.drop_duplicates(subset=['match code', 'balls'])
win_pred_data

,match code,country,over,ball,score,total wickets,win,target,other rating,rating,balls
0,1381452,Germany,0,0,2,0,1,122,58,96,0
1,1381452,Germany,0,1,3,0,1,122,58,96,1
2,1381452,Germany,0,2,4,0,1,122,58,96,2
3,1381452,Germany,0,3,4,0,1,122,58,96,3
4,1381452,Germany,0,4,6,0,1,122,58,96,4
...,...,...,...,...,...,...,...,...,...,...,...
178597,1279382,Isle of Man,14,5,108,4,1,119,0,63,89
178598,1279382,Isle of Man,15,0,114,4,1,119,0,63,90
178599,1279382,Isle of Man,15,1,115,4,1,119,0,63,91
178600,1279382,Isle of Man,15,2,116,4,1,119,0,63,92


In [404]:
#Save the data
#win_pred_data.to_csv(r"../Resources/win_pred_data.csv", index = False)

<h2>Data Analysis</h2>
Now we will get to the real data analysis. First we will check the win/loss ratio in out dataset.

In [408]:
win_pred_data['win'].sum()/ win_pred_data['win'].count()

0.4613615857030591

In [409]:
(win_pred_data['win'].count() - win_pred_data['win'].sum())/ win_pred_data['win'].count()

0.5386384142969409

So we have a total of 79255 rows having win == 1 and 99346 rows having win == 0.
The win/loss ratio is 46.14/53.86, which is pretty much even.